In [1]:
import torch
import pandas as pd
import os
import numpy as np

from scapy.all import *


In [2]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.Load('/home/myk/min0/packet_tokenizer.model')


True

In [3]:
seq_len = 10
ip_length = 20
protocol_length = 60
def get_header_value(packet):
    ip_header_len=min(ip_length,packet[IP].ihl*4)
    ip_byte = bytes(packet.getlayer(IP))[:ip_header_len]
    
    if packet.haslayer(UDP):
        # UDP
        udp_byte = bytes(packet[UDP])[:8]
        protocol_byte = udp_byte
        
    elif packet.haslayer(TCP):
        # TCP
        tcp_header_length = len(packet[TCP]) - len(packet[TCP].payload)
        tcp_byte = bytes(packet[TCP])[:tcp_header_length]
        protocol_byte = tcp_byte
        
    elif packet.haslayer(ICMP):
        icmp_byte = bytes(packet[ICMP])[:8]
        protocol_byte = icmp_byte
        
    else:
        print("no layer")
        return
    
    return ip_byte + protocol_byte

def packet2string(packet):
    bytes = get_header_value(packet)
    
    if bytes is None:
        return
    
    byte_string=''
    for byte in bytes:
        byte_string +=('00'+hex(byte)[2:])[-2:]
    return byte_string
    
def pcap2token(pcap_path, seq_len, tokenizer):
    
    packets = rdpcap(pcap_path)
    
    pad_id, sep_id = sp.PieceToId('[PAD]'),sp.PieceToId('[SEP]')
    token = ['[CLS]']
    
    result = []
    i=0
    while len(result)!=seq_len: 
        packet = packets[i]
        i+=1
        packet_string = packet2string(packet)
        
        # packet byte string으로 가져와서 tokenizer
        if packet_string is None:
            print("here")
            continue
        token.extend(tokenizer.EncodeAsPieces(packet_string))
        id = tokenizer.PieceToId(token)
        
        if(len(id)>70):
            id = id[:70]
        else:
            id.extend([pad_id] * (70 - len(id)))
            
        id.insert(len(id),sep_id)
        result.append(id)
    
    return result 
        

In [4]:
dataset = []
dataset_path = '/home/myk/min0/as'
for dir in os.listdir(dataset_path):
        path = os.path.join(dataset_path,dir)
        for file in os.listdir(path):
            r = pcap2token(os.path.join(path,file),10,sp)
            dataset.append(r)
            

no layer
here
no layer
here


In [10]:
print(torch.tensor(dataset[0]).flatten().shape)

torch.Size([710])


In [9]:
t = torch.zeros([len(dataset),7110])
print(t.shape)

torch.Size([3180, 7110])


In [11]:
t = torch.zeros([len(dataset),710])

for i in range(len(dataset)):
    data = dataset[i]
    for j in range(len(data)):
        t[i] = torch.tensor(data).flatten()
        
print(t.shape)

torch.Size([3180, 710])


In [12]:
np.save("sentencepiece_10_70.npy",t.numpy())